In [ ]:
# Author: Sanjeewa Senanayaka
# Phase 2
# Local Server program that handles login and registration requests

from socket import *
import os
from Crypto import Random
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
import random 
import sys
from _thread import *
import sqlite3
from sqlite3 import Error
from time import time
import struct


serverPort = 62843
serverSocket= socket(AF_INET,SOCK_STREAM)
serverSocket.bind(("",serverPort))
serverSocket.listen(30)
print('Server is listening...')

#Function to create a connection to your SQLite DB
def createConnection():
    database = r"C:\Users\JaY\Documents\UWW- Graduate\Spring 2019\Cryptography and Cloud Security\Project\Phase 1\Users.db"

    try:
        conn = sqlite3.connect(database)
        return conn
    except Error as e:
        print(e)
        
    return None

#Function to create a connection to the cloud server
def createConnectionToCloud():
    serverName = "35.236.106.211"
    serverPort = 3410
    clientSocket = socket(AF_INET,SOCK_STREAM)
    clientSocket.connect((serverName, serverPort))
    return clientSocket

#Function to pad the message
def pad(message):
    length = len(message)
    if(length%16 != 0):
        message += ' '*(16-length%16)
        
    return message


#Function that generates the RSA private and public keys
def RSAKeyGen():
    key = RSA.generate(2048)
    private_key = key.exportKey()
    f = open('mykey.pem','wb')
    f.write(key.exportKey('PEM'))
    f.close()

#Function that checks with the IAM database if credentials match
def loginProcess(method, connectionSocket, session_key, IV):
    if(len(method) > 0):
        AES_object = AES.new(session_key, AES.MODE_CBC, IV)

        status = "Fail"

        username = method.split("\t")[1].strip()
        password = method.split("\t")[2].strip()


        conn = createConnection()
        with conn:
            cur  = conn.cursor()
            cur.execute("SELECT * FROM Credentials WHERE username=? AND password = ?",(username,password,))

            rows = cur.fetchall()
            
            # if the credentials match do the following
            if(len(rows) > 0):
                paid_status = rows[0][3]
                print("Successful login")
                status = "Success"
                
                #generate cookie and prepare message to send
                cookie = random.randint(0, sys.maxsize)
                response = status + "\t" + str(cookie)+"\t"+str(paid_status)
                
                #Connect to cloud and request public key
                print("Connecting to cloud")
                cloudSocket = createConnectionToCloud()
                pub = "publickey"
                padded_pub = pad(pub).encode()
                cloudSocket.send(padded_pub)
                print("waiting for public key")
                public_key = cloudSocket.recv(1024)
                print("public key received")
                
                #Create session key combining current time and random bytes
                curTime = int(time())
                b = struct.pack(">i", curTime)
                session_key_cloud = get_random_bytes(12)+b

                #RSA encrypt the session key and send to the cloud
                keyPub = RSA.importKey(public_key)
                cipher = PKCS1_OAEP.new(keyPub)
                ciphered_session = cipher.encrypt(session_key_cloud)
                print("Sending sessiong key to the cloud: ", session_key_cloud)
                cloudSocket.send(ciphered_session)

                #RSA encrypt the IV and send to the cloud
                IV_cloud = get_random_bytes(16)
                cipher = PKCS1_OAEP.new(keyPub)
                ciphered_IV = cipher.encrypt(IV_cloud)
                cloudSocket.send(ciphered_IV)
                print("Sending IV to the cloud: ", IV_cloud)
                confirmation = cloudSocket.recv(1024).decode()
                print("Confirmation recv: ", confirmation)
                if(confirmation != "Success"):
                    print("Session key too old! Possible replay attack")
                    cloudSocket.close()
                    status = "Fail"
                    response = status + "\t"+ "Session Error between IAM and Cloud"
                    padded_response = pad(response)
                    encrypted_response = AES_object.encrypt(padded_response)
                    connectionSocket.send(encrypted_response)
                    AES_object1 = AES.new(session_key, AES.MODE_CBC, IV)
                    request = connectionSocket.recv(1024)
                    print("REW: ", request)
                    plain_text = AES_object1.decrypt(request).decode("ascii")
                    print("RECEIVED: ", plain_text)
                    loginProcess(plain_text, connectionSocket, session_key, IV)
                else:
                    response_to_cloud = "cookie"+ "\t"+ username+"\t"+str(cookie)+"\t"+str(paid_status)
                    print("sending: ", response_to_cloud)
                    padded_response_to_cloud= pad(response_to_cloud)
                    AES_object1 = AES.new(session_key_cloud, AES.MODE_CBC, IV_cloud)
                    ciphered_padded_response_to_cloud = AES_object1.encrypt(padded_response_to_cloud)
                    cloudSocket.send(ciphered_padded_response_to_cloud)
                    cloudSocket.close()
                    print("to send to client: ", response)
                    padded_response = pad(response)
                    encrypted_response = AES_object.encrypt(padded_response)
                    connectionSocket.send(encrypted_response)
                    connectionSocket.close()
            else:
                status = "Fail"
                response = status + "\t"+ "Your username or password may be wrong"
                padded_response = pad(response)
                encrypted_response = AES_object.encrypt(padded_response)
                connectionSocket.send(encrypted_response)
                AES_object1 = AES.new(session_key, AES.MODE_CBC, IV)
                request = connectionSocket.recv(1024)
                print("REW: ", request)
                plain_text = AES_object1.decrypt(request).decode("ascii")
                print("RECEIVED: ", plain_text)
                loginProcess(plain_text, connectionSocket, session_key, IV)
            


def userTest(plainText, connectionSocket, session_key, IV):
    print(plainText)
    AES_object = AES.new(session_key, AES.MODE_CBC, IV)
    username = str(plainText.split("\t")[1].strip())
    
    conn = createConnection()
    print("1")
    with conn:
        cur  = conn.cursor()
        cur.execute("SELECT username FROM Credentials WHERE username=?",(username,))
    
        rows = cur.fetchall()
        print("2")
        if(len(rows) > 0):
            found = "Fail"
            print(found)
            padded_response = pad(found)
            encrypted_response = AES_object.encrypt(padded_response)
            connectionSocket.send(encrypted_response)
            AES_object1 = AES.new(session_key, AES.MODE_CBC, IV)
            request = connectionSocket.recv(1024)
            plain_text = AES_object1.decrypt(request).decode("ascii")
            userTest(plain_text, connectionSocket, session_key, IV)
            
        else:
            found = "Success"
            print(found)
            padded_response = pad(found)
            encrypted_response = AES_object.encrypt(padded_response)
            connectionSocket.send(encrypted_response)
            registerUser(connectionSocket, session_key, IV)
#         connectionSocket.close()


def registerUser(connectionSocket, session_key, IV):
    AES_object = AES.new(session_key, AES.MODE_CBC, IV)
    request = connectionSocket.recv(1024)
    plainText = AES_object.decrypt(request).decode("ascii")

    AES_object = AES.new(session_key, AES.MODE_CBC, IV)
    username = str(plainText.split("\t")[1].strip())
    password = str(plainText.split("\t")[2].strip())
    first_name = str(plainText.split("\t")[3].strip())
    last_name = str(plainText.split("\t")[4].strip())
    email = str(plainText.split("\t")[5].strip())
    paid = str(plainText.split("\t")[6].strip())
    
    data = (username,password,paid)
    data2 = (first_name,last_name,email)
    conn = createConnection()
    with conn:
        try:
            cloudSocket = createConnectionToCloud()
            pub = "publickey"
            padded_pub = pad(pub).encode()
            cloudSocket.send(padded_pub)
            public_key = cloudSocket.recv(1024)
            curTime = int(time())
            b = struct.pack(">i", curTime)
            session_key_cloud = get_random_bytes(12)+b

            keyPub = RSA.importKey(public_key)
            cipher = PKCS1_OAEP.new(keyPub)
            ciphered_session = cipher.encrypt(session_key_cloud)
            cloudSocket.send(ciphered_session)

            print("Cloud session for registration: ", session_key_cloud)
            IV_cloud = get_random_bytes(16)
            cipher = PKCS1_OAEP.new(keyPub)
            ciphered_IV = cipher.encrypt(IV_cloud)
            cloudSocket.send(ciphered_IV)

            print("IV Cloud for registration: ", IV_cloud)
            confirmation = cloudSocket.recv(1024).decode()
            if(confirmation != "Success"):
                print("Session Key is too old! Possible replay attack! Connection force closing...")
                status = "Fail"
                padded_response = pad(status)
                encrypted_response = AES_object.encrypt(padded_response)
                connectionSocket.send(encrypted_response)
                connectionSocket.close()
                cloudSocket.close()
            else:    
                new_user = "username"+"\t"+username+"\t"+str(0)+"\t"+str(paid)
                padded_new_user= pad(new_user)
                AES_object = AES.new(session_key_cloud, AES.MODE_CBC, IV_cloud )
                ciphered_padded_new_user = AES_object.encrypt(padded_new_user)
                cloudSocket.send(ciphered_padded_new_user)
                cloud_insert = cloudSocket.recv(1024).decode()
                cloudSocket.close()
                print("Insert into cloud: ", cloud_insert)
                if(cloud_insert == "Success"):
                    cur  = conn.cursor()
                    sql = ''' INSERT INTO Credentials(username,password,paid)
                          VALUES(?,?,?) '''
                    cur = conn.cursor()
                    cur.execute(sql, data)
                    conn.commit()

                    sql = ''' INSERT INTO UserInfo(FirstName,LastName,Email)
                          VALUES(?,?,?) '''
                    cur = conn.cursor()
                    cur.execute(sql, data2)
                    conn.commit()

                    status = "Success"
                    padded_response = pad(status)
                    encrypted_response = AES_object.encrypt(padded_response)
                    connectionSocket.send(encrypted_response)
                    connectionSocket.close()
                else:
                    status = "Fail"
                    padded_response = pad(status)
                    encrypted_response = AES_object.encrypt(padded_response)
                    connectionSocket.send(encrypted_response)
                    connectionSocket.close()

                
                    
#                 new_user = "username"+"\t"+username+"\t"+str(0)

                
        except Exception as e:
            print(e)
            status = "Fail"
            padded_response = pad(status)
            encrypted_response = AES_object.encrypt(padded_response)
            connectionSocket.send(encrypted_response)
            connectionSocket.close()
        
def connectionProcessing(connectionSocket):
    #Send the public key to the connected client
    f = open('mykey.pem','r')
    key = RSA.importKey(f.read())
    keyPub = key.publickey().exportKey()   
    connectionSocket.send(keyPub)
    
    #Receive the session key from the client
    session_key = connectionSocket.recv(1024) 
    private_key = RSA.importKey(key.exportKey())
    cipher_rsa = PKCS1_OAEP.new(private_key)
    session_key = cipher_rsa.decrypt(session_key)
    
    time_stamp = struct.unpack(">i",session_key[12:16])
    time_stamp = time_stamp[0]
    current_time = int(time())
    time_difference = current_time - time_stamp
    #Receive the IV from the client
    IV = connectionSocket.recv(1024) 
    private_key = RSA.importKey(key.exportKey())
    cipher_rsa = PKCS1_OAEP.new(private_key)
    IV = cipher_rsa.decrypt(IV)
    
    if time_difference > 2:
        fail = "Fail"
        fail = fail.encode()
        connectionSocket.send(fail)
        
    else:
        success ="Success"
        success = success.encode()
        connectionSocket.send(success)
        AES_object = AES.new(session_key, AES.MODE_CBC, IV)
        request = connectionSocket.recv(1024)
        plain_text = AES_object.decrypt(request).decode("ascii")

        method = plain_text.split("\t")[0].strip()
        print("method is: " + method)

        if(method == "Login"):
            print("method in: " + method)
            loginProcess(plain_text, connectionSocket, session_key, IV)

        elif(method == "Username"):
            print("method in: " + method)
            userTest(plain_text, connectionSocket, session_key, IV)

        elif(method == "Registration"):
            registerUser(plain_text,connectionSocket, session_key, IV)

def main():        
    RSAKeyGen()
    while 1:
        connectionSocket,addr = serverSocket.accept()
        start_new_thread(connectionProcessing, (connectionSocket,))
    
main()